In [2]:
!docker login -u mariembouhadda -p sS58169362

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/jupyter/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
%%writefile Dockerfile
FROM tensorflow/tensorflow
COPY ./ ./

Overwriting Dockerfile


In [12]:
!docker build -t mariembouhadda/hello-world .

Sending build context to Docker daemon  10.24kB
Step 1/2 : FROM tensorflow/tensorflow
 ---> 67f912b32b47
Step 2/2 : COPY ./ ./
 ---> 95fe3300b602
Successfully built 95fe3300b602
Successfully tagged mariembouhadda/hello-world:latest


In [13]:
!docker push mariembouhadda/hello-world

Using default tag: latest
The push refers to repository [docker.io/mariembouhadda/hello-world]

dc6cd778: Preparing 
044609ed: Preparing 
2430415b: Preparing 
de7ae625: Preparing 
272de80e: Preparing 
471b421b: Preparing 
4d2a8fe6: Preparing 
f09b441c: Preparing 
dceb4c77: Mounted from mariembouhadda/titanic-kubeflow latest: digest: sha256:a3672582b60cd25217e6a93ae3750e59a78142305553b31027b479f3c0136795 size: 2208


In [14]:
import kfp
import kfp.components as comp

In [15]:
@kfp.dsl.component
def hello_world():
    # Defining component configuration
    hello_component = kfp.dsl.ContainerOp(
        name='hello-world',
        image='docker.io/mariembouhadda/hello-world',
        command=['python', 'hello_world.py'],
        )
    return hello_component

In [16]:
@kfp.dsl.pipeline(
  name="hello world script",
  description="hello world script"
)
def hi():
    hello = hello_world()
    hello.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [18]:
kfp.compiler.Compiler().compile(hi, 'hello_world.zip')

In [19]:
client = kfp.Client(host='https://b5667049ffd46dc-dot-us-central1.pipelines.googleusercontent.com')

In [20]:
EXPERIMENT_NAME = 'hello world'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

In [21]:
run = client.run_pipeline(experiment.id, 'hello world', 'hello_world.zip')